In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/house_train.csv',index_col=0)
test = pd.read_csv('/content/house_test.csv',index_col=0)

In [ ]:
train = train.fillna(0)

In [ ]:
cat_features = list(set(train.columns) - set(train.describe().columns))

In [ ]:
X = train.drop('SalePrice',axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, train['SalePrice'], test_size=0.2, random_state=42)

In [ ]:
model = CatBoostRegressor(
    iterations=1e4, depth=5,
    loss_function='RMSE', verbose=True,metric_period=int(1e3),
    cat_features=cat_features)

model.fit(X_train, y_train,eval_set=(X_test,y_test))

In [ ]:
importances = model.get_feature_importance()
feature_importances = pd.Series(importances, index=X_train.columns).sort_values(ascending=False).reset_index()

In [ ]:
trimmed_feat = list(feature_importances[feature_importances[0] >= 2]['index'])
cat_features = list(set(train[trimmed_feat].columns) - set(train[trimmed_feat].describe().columns))

In [ ]:
model = CatBoostRegressor(
    iterations=1e4, depth=5,
    loss_function='RMSE', verbose=True,metric_period=int(1e3),
    cat_features=cat_features)

model.fit(X_train[trimmed_feat], y_train, eval_set=(X_test[trimmed_feat],y_test))

In [ ]:
predictions = model.predict(X_test[trimmed_feat])